In [ ]:
import h5py
import matplotlib.pyplot as plt
import mpl_lego.colors as colors
import neuropacks as packs
import numpy as np
import os

from mpl_lego.ellipse import plot_cov_ellipse
from mpl_lego.style import use_latex_style
from neurobiases import analysis
from neurobiases.utils import all_correlations
from mpl_lego.labels import bold_text
from pyprojroot import here

%matplotlib inline

In [ ]:
use_latex_style()

In [ ]:
ccycle = colors.get_default_ccycle()

In [ ]:
# Base paths
base_path = here('data')
# PVC11 path
pvc11_path = os.path.join(base_path, 'data_monkey1_gratings.mat')
ecog_path = os.path.join(base_path, 'r32_b7.mat')
hc6_path = os.path.join(base_path, 'Bon')
# Create neuropack
pvc11 = packs.PVC11(data_path=pvc11_path)
ecog = packs.ECOG(data_path=ecog_path)
hc6 = packs.HC6(directory=hc6_path)

In [ ]:
# Obtain spike times from PVC
pvc11_spike_times = pvc11.events.ravel()

In [ ]:
# Get spike counts for each neuron
counts = np.zeros_like(pvc11_spike_times)

for neuron in range(counts.size):
    counts[neuron] = pvc11_spike_times[neuron].size

# What are the neurons with the highest number of counts?
pvc_highest = np.flip(np.argsort(counts))

In [ ]:
# Obtain ECOG responses to plot
hg_responses = np.mean(ecog.responses[:, :, :, :, :, 2], axis=1).reshape((101, -1)).T
ecog_highest = np.flip(np.argsort(np.max(hg_responses, axis=1)))

In [ ]:
# Obtain spike times from HC6
hc6_spike_times = hc6.spike_times[3][1]
# Get minimum time
min_times = [np.min(hc6_spike_times[idx])
             for idx in range(len(hc6_spike_times))
             if len(hc6_spike_times[idx]) > 0]
overall_min = np.min(min_times)

In [ ]:
# Get spike counts for each neuron
counts = np.zeros(len(hc6_spike_times))

for neuron in range(counts.size):
    counts[neuron] = hc6_spike_times[neuron].size

# What are the neurons with the highest number of counts?
hc6_highest = np.flip(np.argsort(counts))

In [ ]:
# Calculate noise correlation distribution for each dataset
X_pvc11 = pvc11.get_response_matrix()
stimuli_pvc11 = pvc11.get_design_matrix()
ncs_pvc11 = all_correlations(X_pvc11, stimuli_pvc11)

X_ecog = ecog.get_response_matrix(bounds=(40, 60), band='HG')
stimuli_ecog = ecog.get_design_matrix(form='frequency')
ncs_ecog = all_correlations(X_ecog, stimuli_ecog)

X_hc6 = hc6.get_response_matrix(day=3, epoch=1, bin_width=0.5)
x, y = hc6.get_binned_positions(day=3, epoch=1, bin_width=0.5)
XY_hc = np.vstack((x, y)).T
x_idx = np.digitize(XY_hc[:, 0], bins=np.linspace(35, 125, 10))
y_idx = np.digitize(XY_hc[:, 1], bins=np.linspace(55, 155, 10))
stimuli_hc6 = 10 * y_idx + x_idx
ncs_hc6 = all_correlations(X_hc6, stimuli_hc6)

In [ ]:
"""
Figure 1
"""
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
plt.subplots_adjust(wspace=0.4, hspace=0.5)

# Figure settings
spike_color = 'black'
spike_size = 100
spike_size_hc = 80
nc_histtype = 'step'
hist_color = 'black'
hist_lw = 2
axis_tick_label_size = 13
axis_label_size = 15

"""
Figure 1b
Spike raster for PVC
"""
# Iterate over neurons in visual cortex, plotting spikes
for idx, n_idx in enumerate(pvc_highest[:20]):
    times = pvc11_spike_times[n_idx]
    axes[0, 0].scatter(1000 * times,
                       idx * np.ones_like(times),
                       marker='|',
                       color=spike_color,
                       s=spike_size)

axes[0, 0].set_xlim([0, 400])
axes[0, 0].set_xticks([0, 100, 200, 300, 400])

"""
Figure 1c
Noise correlation for PVC
"""
axes[0, 1].hist(ncs_pvc11, histtype=nc_histtype, color=hist_color, lw=hist_lw)

"""
Figure 1e
ECOG Traces
"""
# Iterate over electrodes in ECOG, plotting HG traces
for idx, e_idx in enumerate(ecog_highest[0:10000:500]):
    # Normalize each trace
    ecog_response = hg_responses[e_idx]
    ecog_response /= np.max(ecog_response)
    # Obtain times
    times = np.arange(ecog_response.size) * 1000 / 200
    # Plot, with spacing for each electrode
    axes[1, 0].plot(times, 1.2 * idx + ecog_response, color='black')

axes[1, 0].set_xlim([0, 400])
axes[1, 0].set_xticks([0, 100, 200, 300, 400])

"""
Figure 1f
Noise correlation for ECOG
"""
axes[1, 1].hist(ncs_ecog, histtype='step', color='black', lw=2)

"""
Figure 1h
Spike raster for HC6
"""
# Iterate over neurons in hippocampus, plotting spikes
for idx, n_idx in enumerate(hc6_highest[20:40]):
    times = hc6_spike_times[n_idx]
    axes[2, 0].scatter(times - overall_min,
                       idx * np.ones_like(times),
                       marker='|',
                       color=spike_color,
                       s=spike_size_hc)

"""
Figure 1i
Noise correlation for HC6
"""
axes[2, 1].hist(ncs_hc6, histtype='step', color='black', lw=2)

# Set response plot labels
for ax in axes[:, 0]:
    ax.set_xlabel(bold_text('Time since onset (ms)'), fontsize=axis_label_size)
    ax.set_ylabel(bold_text('Unit'), fontsize=axis_label_size)
    ax.set_yticks([])
axes[2, 0].set_xlabel(bold_text('Time since onset (s)'), fontsize=axis_label_size)

# Set noise correlation plot labels
for ax in axes[:, 1]:
    ax.set_xlim([-1, 1])
    ax.set_xticks([1, -0.5, 0, 0.5, 1.])
    ax.set_xlabel(bold_text('Noise Correlation'), fontsize=axis_label_size)
    ax.set_ylabel(bold_text('Frequency'), fontsize=axis_label_size)
    
# Set tick label sizes
for ax in axes.ravel():
    ax.tick_params(labelsize=axis_tick_label_size)
    
plt.savefig('figure1_sub.pdf', bbox_inches='tight')